In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from array import *
!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  !pip install tf-nightly
except Exception:
  pass
import tensorflow_datasets as tfds
import tensorflow as t

import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

  #el imbd contiene 50,000 reseñas de películas de la base de datos de películas de Internet

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [3]:

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)
#descargo el dataset y tomo el 60% para entrenar, validar y probar 
dataset, info = tfds.load(name="imdb_reviews", with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNMRF1Y/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNMRF1Y/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNMRF1Y/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
# 0 es una crítica negativa y 1 es una crítica positiva.
#train_labels_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3]) #<- probamos

# modelo de incrustación de texto previamente entrenado

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
#La primera capa es una capa de TensorFlow Hub. Esta capa mapea una oración en su vector de incrustación, divide la oración en tokens, incrusta cada token y luego combina la incrustación. Las dimensiones resultantes son: (num_examples, embedding_dimension).
#Este vector de salida de longitud fija se canaliza a través de una Densecapa completamente conectada ( ) con 16 unidades ocultas.
#Valor entre 0 y 1 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
#modelo para usar un optimizador y una función de pérdida:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
#Entrenamos el modelo
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 214ms/step - loss: 1.2194 - accuracy: 0.4674 - val_loss: 0.7963 - val_accuracy: 0.4402
Epoch 2/20
30/30 [==============================] - 6s 196ms/step - loss: 0.7366 - accuracy: 0.5009 - val_loss: 0.6879 - val_accuracy: 0.5691
Epoch 3/20
30/30 [==============================] - 6s 194ms/step - loss: 0.6549 - accuracy: 0.6089 - val_loss: 0.6336 - val_accuracy: 0.6393
Epoch 4/20
30/30 [==============================] - 6s 199ms/step - loss: 0.6094 - accuracy: 0.6670 - val_loss: 0.5988 - val_accuracy: 0.6807
Epoch 5/20
30/30 [==============================] - 6s 212ms/step - loss: 0.5730 - accuracy: 0.7057 - val_loss: 0.5658 - val_accuracy: 0.7160
Epoch 6/20
30/30 [==============================] - 6s 204ms/step - loss: 0.5337 - accuracy: 0.7446 - val_loss: 0.5308 - val_accuracy: 0.7519
Epoch 7/20
30/30 [==============================] - 6s 207ms/step - loss: 0.4941 - accuracy: 0.7771 - val_loss: 0.4933 - val_accuracy: 0.7786
Epoch 

In [10]:
#evaluamos el modelo
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.319
accuracy: 0.864


In [0]:
temp_list = [] 
temp_list.append("i love this movie")
temp_list.append("i love this movie")


In [12]:
predictions = model.predict(temp_list)
predicted_labels = ""
CLASS_NAMES = ['good','bad']
for i in (-predictions[0]).argsort()[:2]:
  predicted_labels += CLASS_NAMES[i] + " (" + str(np.around(predictions[0][i], decimals=2)) + ")\n"

print('\nLa prediccion es:\n', predicted_labels)


La prediccion es:
 good (0.73)

